## Aims

1. Understand the basics of AMR testing, the data formats and limitations
2. Understand the basics of sequencing methods and the types of data available to work with
3. Learn where to find publicly available data and how to access it
4. Perform initial data processing to transform into a usable format for further explorations

### Imports

In [2]:
import bz2
import os
import subprocess
import requests

import pandas as pd

## 1. Explore BV-BRC

Here is a link to the BV-BRC (Bacterial and Viral Bioinformatics Resource Center) website: https://www.bv-brc.org/

Take some time to look around the website and think about the following: What data looks relevant for our modeling task and how will we access it?

<div class="question" style="color: #534646; background-color: #ffdfa3; padding: 1px; border-radius: 5px;">

##### Q. Any areas that stand out as interesting?

</div>

## 2. Download AMR Phenotypes

For this project we'll focus on `Escherchia coli` - one of the most common pathogens infecting humans, and one which is well studied and categorized.

To start with we'll look at collecting our target data (AMR Phenotype) and work backwards from there to collect all the data we'll need for modeling.

#### 2a. Download Data 

We're using subprocess here to allow us to call a command line tool directly in our notebook - if WGET isn't installed on your system you can also skip the below and grab the file from the `data/` folder in the GitHub repo

In [3]:
genome_amr_ftp = 'ftp://ftp.bvbrc.org/RELEASE_NOTES/PATRIC_genomes_AMR.txt'
subprocess.run(['wget', genome_amr_ftp], capture_output=False, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

CompletedProcess(args=['wget', 'ftp://ftp.bvbrc.org/RELEASE_NOTES/PATRIC_genomes_AMR.txt'], returncode=0)

#### 2b. Review Downloaded Data

Lets take a look at the data we just downloaded:

In [4]:
amr_data = pd.read_csv('PATRIC_genomes_AMR.txt', sep='\t', low_memory=False)
print(f'Data shape: {amr_data.shape}\n')
amr_data.head(5)

Data shape: (750541, 16)



,genome_id,genome_name,taxon_id,antibiotic,resistant_phenotype,measurement,measurement_sign,measurement_value,measurement_unit,laboratory_typing_method,laboratory_typing_method_version,laboratory_typing_platform,vendor,testing_standard,testing_standard_year,source
0,32002.4,Achromobacter denitrificans strain USDA-ARS-US...,32002.0,ampicillin,NaN,==16,==,16,mg/L,Broth dilution,NaN,Sensititre,Trek Diagnostic Systems,CLSI,NaN,NaN
1,32002.4,Achromobacter denitrificans strain USDA-ARS-US...,32002.0,ceftiofur,NaN,>8,>,8,mg/L,Broth dilution,NaN,Sensititre,Trek Diagnostic Systems,CLSI,NaN,NaN
2,32002.4,Achromobacter denitrificans strain USDA-ARS-US...,32002.0,chlortetracycline,NaN,==8,==,8,mg/L,Broth dilution,NaN,Sensititre,Trek Diagnostic Systems,CLSI,NaN,NaN
3,32002.4,Achromobacter denitrificans strain USDA-ARS-US...,32002.0,clindamycin,NaN,>16,>,16,mg/L,Broth dilution,NaN,Sensititre,Trek Diagnostic Systems,CLSI,NaN,NaN
4,32002.4,Achromobacter denitrificans strain USDA-ARS-US...,32002.0,danofloxacin,NaN,==1,==,1,mg/L,Broth dilution,NaN,Sensititre,Trek Diagnostic Systems,CLSI,NaN,NaN


In [5]:
escherichia_coli_tax_id = 562

amr_data = amr_data[amr_data.taxon_id == escherichia_coli_tax_id]
amr_data.head(5)

,genome_id,genome_name,taxon_id,antibiotic,resistant_phenotype,measurement,measurement_sign,measurement_value,measurement_unit,laboratory_typing_method,laboratory_typing_method_version,laboratory_typing_platform,vendor,testing_standard,testing_standard_year,source
38904,562.964200,Citrobacter freundii 3553,562.0,colistin,NaN,2,NaN,2,mg/L,Broth dilution,NaN,NaN,Merlin Diagnostika,EUCAST,2019.0,NaN
72978,562.100421,Escherichia coli 00116690-7bb9-11e9-a8d3-68b59...,562.0,ampicillin,Susceptible,NaN,NaN,NaN,NaN,Disk diffusion,NaN,NaN,NaN,EUCAST,NaN,NaN
72979,562.100421,Escherichia coli 00116690-7bb9-11e9-a8d3-68b59...,562.0,cefotaxime,Susceptible,NaN,NaN,NaN,NaN,Disk diffusion,NaN,NaN,NaN,EUCAST,NaN,NaN
72980,562.100421,Escherichia coli 00116690-7bb9-11e9-a8d3-68b59...,562.0,ceftazidime,Susceptible,NaN,NaN,NaN,NaN,Disk diffusion,NaN,NaN,NaN,EUCAST,NaN,NaN
72981,562.100421,Escherichia coli 00116690-7bb9-11e9-a8d3-68b59...,562.0,cefuroxime,Resistant,NaN,NaN,NaN,NaN,Disk diffusion,NaN,NaN,NaN,EUCAST,NaN,NaN


In [6]:
amr_data.value_counts(['antibiotic', 'resistant_phenotype']).sort_index().head(10)

antibiotic                   resistant_phenotype
amikacin                     Intermediate             83
                             Resistant                26
                             Susceptible            2075
amoxicillin                  Intermediate             77
                             Resistant              1042
                             Susceptible             734
amoxicillin/clavulanic acid  Intermediate             94
                             Resistant              1216
                             Susceptible            2766
ampicillin                   Intermediate             13
Name: count, dtype: int64

Far too many to deal with - lets subset down to just a single antibiotic of interest. Much like species selection we could expand out to more antibiotics be for now we'll be focusing on just: **cefepime**

Further details on cefepime can be found on the NCBI's website: https://www.ncbi.nlm.nih.gov/books/NBK542232/

At a high level:
1. Cefepime is a fourth generation cephalosporin in the beta-lactam class
2. Cefepime acts by binding to the bacteria and preventing cell wall formation
3. Escherichia coli is know to develop resistance in some cases to do beta-lactamase production

#### 2c. Subset to Cefepime Antibiotic


In [7]:
amr_data = amr_data[amr_data.antibiotic == 'cefepime']

In [8]:
amr_data.fillna('missing').value_counts('resistant_phenotype')

resistant_phenotype
Susceptible                   865
Resistant                     161
missing                       108
Intermediate                   56
Susceptible-dose dependent      1
Name: count, dtype: int64

In [9]:
# Drop missing targets
amr_data = amr_data[pd.notnull(amr_data.resistant_phenotype)]

This looks like a good starting point for building out our modeling task - genomes are large and so working with ~1,000 genomes will be around the limit we'll be able to handle during this workshop

### 3. Download Genomes

So we now have a set of AMR phenotypes (our targets) but we don't have any training data yet

Lets collect our genomic data now from BV-BRC. We want to work with sequencing data directly so we'll look to download

#### 3a. Collect Genome Metadata

First lets pull all the details of the available genomes - similar to how we pulled the AMR data (again if WGET isn't installed on your system you can access the file through the `data/` folder)

In [84]:
genome_genome_summary_ftp = 'ftp://ftp.bvbrc.org/RELEASE_NOTES/genome_summary'
subprocess.run(['wget', genome_genome_summary_ftp], capture_output=False, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

CompletedProcess(args=['wget', 'ftp://ftp.bvbrc.org/RELEASE_NOTES/genome_summary'], returncode=0)

In [10]:
genome_summary = pd.read_csv('genome_summary', sep='\t', low_memory=False)
print(f'Data shape: {genome_summary.shape}\n')
genome_summary.head(5)

Data shape: (1019181, 20)



,genome_id,genome_name,taxon_id,genome_status,genome_length,gc_content,contig_l50,contig_n50,chromosomes,plasmids,contigs,patric_cds,refseq_cds,trna,rrnacoarse_consistency,fine_consistency,checkm_completeness,checkm_contamination,genome_qualitydate_created,date_modified
0,469009.4,"""'Brassica napus' phytoplasma strain TW1""",469009,WGS,743598,27.23313,1.0,373899.0,NaN,NaN,5,1073.0,705.0,27.0,NaN,98.2,100.0,10.3,NaN,2018-07-15T20:38:31.410Z
1,1309411.5,"""'Deinococcus soli' Cha et al. 2014 strain N5""",1309411,Complete,3236984,70.20000,1.0,3236984.0,1.0,NaN,1,3197.0,2944.0,50.0,NaN,98.1,96.9,1.0,NaN,2016-01-17T16:02:31.274Z
2,1123738.3,"""'Echinacea purpurea' witches'-broom phytoplas...",1123738,WGS,545427,23.89000,4.0,46697.0,NaN,NaN,28,535.0,433.0,26.0,NaN,94.6,84.0,6.4,NaN,2016-01-17T17:13:35.332Z
3,551115.6,"""'Nostoc azollae' 0708""",551115,Complete,5486145,38.30000,1.0,5354700.0,1.0,2.0,3,7014.0,3651.0,0.0,NaN,99.2,99.3,0.7,NaN,2015-03-16T03:17:09.594Z
4,1856298.3,"""'Osedax' symbiont bacterium Rs2_46_30_T18 str...",1856298,WGS,4021833,45.76000,63.0,19390.0,NaN,NaN,365,4118.0,3690.0,59.0,NaN,88.3,98.7,3.1,NaN,2017-07-22T04:04:17.224Z


Even larger! 

This is why we started with the AMR data - we can use it to subset down to just samples which are useful to our modeling problem 

In [11]:
relevant_genomes = genome_summary[genome_summary.genome_id.isin(amr_data.genome_id.unique())]

In [12]:
relevant_genomes.value_counts('genome_status')

genome_status
WGS           1103
Complete        56
Deprecated       1
Name: count, dtype: int64

In [13]:
# Drop deprecated genomes & duplicates
relevant_genomes = relevant_genomes[relevant_genomes.genome_status != 'Deprecated']
relevant_genomes = relevant_genomes.drop_duplicates('genome_id')

In [14]:
relevant_genomes.value_counts('genome_status')

genome_status
WGS         1031
Complete      48
Name: count, dtype: int64

#### 3b. Download Genomes

Now we have the metadata we'll need to download the genomes themselves. As we can see above, all of our data is either WGS or Complete - in our case these are equivalent and represent that the data has been sequenced, assembled and uploaded to BV-BRC.

We can download these assemblies using command line tools again - this time we'll need to download each genome one at a time.

The FTP link we'll be using is of the format (.FNA = FASTA format):
```
ftp://ftp.bvbrc.org/genomes/$genome_id/$genome_id.fna
```

where `$genome_id` = our genome_id for each sample

In [15]:
# Iterate through the genome IDs and download each one
for i, genome_id in enumerate(relevant_genomes.genome_id[0:5]):
    sample_assembly_path = f'ftp://ftp.bvbrc.org/genomes/{genome_id}/{genome_id}.fna'
    subprocess.run(
        ['wget', sample_assembly_path, '-P', '../data/genomes/'], 
        capture_output=False, 
        stdout=subprocess.DEVNULL, 
        stderr=subprocess.STDOUT
    )

##### Notes:

- Downloading all 1000 fasta files would take 15-25 minutes to complete the download process for all files
- Each assembly file is ~5MB in size and there are 1,000 hence our dataset will end up at ~5GB in size
- Instead of waiting for the full download the files have already been downloaded and stored for you in the course data directory under `genomes/`

### 4. Process CARD Data

5GB worth of sequencing data is going to be very difficult to work with, luckily we chose a well documented species/drug combination in Escherichia coli + cefepime and hence we can leverage existing knowledge of relevant resistance mechanisms.

In looking for only known resistance genes (portions of the genome which are known to confer resistances) we can subset each genome down to a much smaller set of sequences to work with.

To do so we're going to need to build up a FASTA of useful genes which we can use to subset our samples - this is where CARD comes in!

#### 4a. Download CARD Data

In [16]:
card_data_url = 'https://card.mcmaster.ca/download/0/broadstreet-v3.2.9.tar.bz2'
subprocess.run(
    ['wget', card_data_url, '-P', '../data/'], 
    capture_output=False, 
    stdout=subprocess.DEVNULL, 
    stderr=subprocess.STDOUT
)
os.rename('../data/broadstreet-v3.2.9.tar.bz2', '../data/card_data.tar.bz2')

This file needs decompressing - the easiest way will be to navigate to the file in your file browser and unzip it using your OS. You should see a `card_data.tar.bz2` file within your data folder, unzip this and take a look around.

#### 4b. Generate a Single combined CARD dataset

CARD splits the different types of resistance mechanisms out into different FASTA files (e.g. homolog, variant, knockout).

These are actually really important when using the CARD data, for example:
1. Homologs are inherited genes or sequences that if present confer resistance
2. Variants are references sequences that when a mutation occurs within specific locations, a resistance is conferred

Clearly 1 and 2 are very different: finding a sequencing for 1 mean we expect to see an increase in resistance whereas seeing a sequencing from 2 is meaningless without also seeing the specific mutations.

The way we're going to use this data however is to align to our sample's genomes and leave some flexbility in the alignment (see the W2_exercise) so we can then use Machine Learning to pick out the important features in either case.

To start with - lets combine all our "nucleotide" models (using ACGT DNA bases) into a single FASTA to make it easier to work with.

In [17]:
# Loop through out nucleotide FASTA files and join
fasta_models = []
for file_name in os.listdir('../data/card_data/'):
    if file_name.startswith('nucleotide_'):
        print(file_name)
        with open(f'../data/card_data/{file_name}') as f:
            fasta = f.readlines()
            print(len(fasta))
            fasta_models.extend(fasta)

# Write back out as combined
with open(f'../data/card_data/nucleotide_combined_model.fasta', 'w') as f:
    f.writelines(fasta_models)

nucleotide_fasta_protein_variant_model.fasta
376
nucleotide_fasta_protein_overexpression_model.fasta
30
nucleotide_fasta_protein_knockout_model.fasta
36
nucleotide_fasta_rRNA_gene_variant_model.fasta
174
nucleotide_fasta_protein_homolog_model.fasta
9610


### 5. Save out all our Data


In [18]:
# Save AMR data
amr_data.to_csv('../data/amr_data.csv')

# Save Genome summary
relevant_genomes.to_csv('../data/e_coli_summary_data.csv')